# NLE Assessed Coursework 3: Question 3

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 3 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=198735 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/Joe/Documents/Python Scripts/resources/resources')
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from sussex_nltk.corpus_readers import ReutersCorpusReader
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import math
import spacy
#nlp=spacy.load('en')
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg
import operator

Sussex NLTK root directory is /Users/Joe/Documents/Python Scripts/resources/resources


## Question 3: Named Entity Recognition and Linking (25 marks)

The code below will run the SpaCy system on the text from Persuasion by Jane Austen.  `mysample` contains a 50% sample which is unique to your candidate number.

In [3]:
#Do NOT change the code in this cell.

#preparing corpus

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()


def get_sample(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=int(len(sentslist)/2)
    return sentslist[testsize:]
    
persuasion=clean_text(gutenberg.raw('austen-persuasion.txt'))
nlp_persuasion=list(nlp(persuasion).sents)

mysample=get_sample(nlp_persuasion)

In [4]:
type(mysample[0])

spacy.tokens.span.Span

a) **Write code** and **extract**:
* the 30 most common strings referring to PEOPLE in `mysample`.
* the 30 most common strings referring to PLACES in `mysample`.

\[6 marks\]

In [5]:
def make_tag_lists(sents):
    tokens=[]
    pos=[]
    ner=[]
    for sent in sents:
        for t in sent:
            tokens.append(t.text)
            pos.append(t.pos_)
            ner.append(t.ent_type_)
    return tokens,pos,ner

def extract_entities(tokenlist,taglist,tagtype):
    
    entities={}
    inentity=False
    for i,(token,tag) in enumerate(zip(tokenlist,taglist)):
        if tag==tagtype:
            if inentity:
                entity+=" "+token
            else:
                entity=token
                inentity=True
        elif inentity:
            entities[entity]=entities.get(entity,0)+1
            inentity=False
    return entities  

In [6]:
tokens,tag,refer = make_tag_lists(mysample)
#print(tag_list)
people=extract_entities(tokens,refer,"PERSON")
top_people=sorted(people.items(),key=operator.itemgetter(1),reverse=True)[:30]
print("30 Most common strings referring to PEOPLE:")
print(top_people)
def add_to_dict(dict1,dict2):
    for key in dict2.keys():
        if dict1.get(key,0) != 0:
            dict1[key] = dict1[key] + dict2[key]
        else:
            dict1[key] = dict2[key]
    return dict1
places=extract_entities(tokens,refer,"LOC")
temp = extract_entities(tokens,refer,"GPE")
places = add_to_dict(places,temp)
temp2 = extract_entities(tokens,refer,"FAC")
places = add_to_dict(places,temp2)
top_places=sorted(places.items(),key=operator.itemgetter(1),reverse=True)[:30]
print("30 Most common strings referring to PLACES:")
print(top_places)

30 Most common strings referring to PEOPLE:
[('Anne', 212), ('Elliot', 113), ('Mary', 73), ('Captain Wentworth', 62), ('Charles', 61), ('Walter', 59), ('Lady Russell', 50), ('Elizabeth', 46), ('Wentworth', 26), ('Louisa', 24), ('Musgrove', 19), ('Mrs Musgrove', 18), ('Harville', 17), ('Mrs Clay', 17), ('Captain Benwick', 16), ('Smith', 15), ('Charles Hayter', 14), ('Wallis', 14), ('Lady Dalrymple', 14), ('Mrs Smith', 13), ('Musgroves', 13), ("Lady Russell 's", 12), ('Benwick', 12), ('Captain Harville', 9), ('Mr Shepherd', 8), ('Charles Musgrove', 7), ('Lyme', 7), ('Anne Elliot', 7), ('Mrs Croft', 7), ("Captain Wentworth 's", 7)]
30 Most common strings referring to PLACES:
[('Uppercross', 27), ('Kellynch', 11), ('Camden Place', 10), ('Lyme', 7), ('Monkford', 6), ('the Great House', 5), ('the West Indies', 4), ('Plymouth', 3), ('London', 3), ('the Octagon Room', 3), ('Cape', 2), ('the Concert Room', 2), ('Mrs Musgrove', 2), ('Uppercross Cottage', 2), ("Captain Wentworth 's", 2), ('Rivers

b) Making reference to specific examples from the text in `mysample`, **discuss** the different types of errors made by the named entity recogniser. \[6 marks\]

Entities that are given a type may have multiple different types. For example: The entity *'Mrs. Wallis'* can be a person or a place in the context of the text in `mysample`. This could be solved by looking at the surrounding text in a context window in order to gain an idea as to which the entity is referring to in the sentence. There maybe variation of the same entity that is recognised as a different type by the named entity recogniser. For example: *'Captain Benwick'* and *'Benwick'* are variation of a person. This can be solved by having a database that contains named entities and their variations. An entity maybe of the same type, for example: there are multiple places in the world that could be called *'Rivers Street'*. This can therefore lead to ambiguity and an understanding about which *'Rivers street'* it is referring to could help to correctly establish the difference between them. 

c) **Design** and **implement** a system to track the locations of characters throughout the story.  For a given PERSON named entity, your system should return a list of time-ordered LOCATIONS for that character.  Test your system using the complete text of "Persuasion" (**not** `mysample`) for at least 3 major characters.   \[13 marks\]

The system will look through the text and relate person to location in the sentence that it is currently in, using the punctuation and entity tags given by the nlp function, denoted by punctuation that stops a sentence *'.','!','?'*. The input being the person we are looking for and the tokens that we are looking through, output is the person and a time ordered list i.e. appended to the end of the list of the locations that person has been.

In [57]:
def person_locations(tokens, person):
    locations = []
    location_change = 0
    current_location = ''
    current_entity = ''
    for token in tokens:
        if token.ent_type_ in ('LOC','FAC','GPE') and location_change == 1:
            if token.ent_type_ == current_entity:
                current_location += ' ' + token.text
                current_entity = token.ent_type_
            else:
                if current_location != '':
                    locations.append(current_location[1:])
                current_location = ' ' + token.text
                current_entity = token.ent_type_
        else:
            if token.text == person:
                location_change = 1
            elif token.text in ('.','!','?'):
                location_change = 0          
            if current_location != '':
                locations.append(current_location[1:])    
            current_location = ''
            current_entity = ''            
    return {person : locations}

text = nlp(persuasion)
print(person_locations(text,'Anne'))
print(person_locations(text,'Elliot'))
print(person_locations(text,'Mary'))

{'Anne': ['London', 'Kellynch', 'Uppercross Cottage', 'Uppercross Cottage', 'Uppercross Cottage', 'Kellynch', "Mrs Musgrove 's", 'England', "Captain Wentworth 's", 'Uppercross', 'Uppercross', 'the Mansion House', 'Mr', 'Uppercross', 'Camden Place', 'Kellynch Hall', 'Uppercross', 'Uppercross', 'Camden Place', 'Camden Place', 'Westgate Buildings', 'Westgate Buildings', 'England', 'Ireland', 'Kellynch', 'Uppercross', 'Lyme', 'Camden Place', 'Milsom Street', 'Camden Place', 'Pulteney Street', 'the Mr Wentworth', 'Monkford', 'Camden', 'Camden Place', 'Uppercross', 'Mrs Musgrove', 'Rivers Street', 'Uppercross Hall', "Captain Wentworth 's"]}
{'Elliot': ['Kellynch Hall', 'Somersetshire', 'Esq', 'Esq .', 'London', 'Sidmouth', 'Uppercross', 'Camden Place', 'Camden Place', 'Kellynch', 'the Dowager Viscountess', 'Westgate Buildings', 'England', 'Ireland', 'Kellynch', 'Rivers Street']}
{'Mary': ['Kellynch', 'Uppercross Cottage', 'Uppercross Cottage', 'the Great House', 'the Great House', 'the Great

In [2]:
##This code will word count all of the markdown cells in the notebook saved at filepath
##Running it before providing any answers shows that the questions have a word count of 1202

import io
from nbformat import current

filepath="a3_3.ipynb"
question_count=211

with io.open(filepath, 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')

word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print("Submission length is {}".format(word_count-question_count))

Submission length is 238
